In [5]:
import scipy.io
import os
import numpy as np


# Define class map globally
class_map = {'Normal': 0, 'Bearing Fault': 1, 'Misalignment': 2, 'Rotor Imbalance': 3}


def simulate_misalignment(signal, fs=10000):
    t = np.arange(len(signal)) / fs
    return signal + 0.1 * np.sin(2 * np.pi * 1 * t)  # 1 Hz perturbation

def simulate_rotor_imbalance(signal, fs=10000):
    t = np.arange(len(signal)) / fs
    return signal + 0.1 * np.sin(2 * np.pi * 25 * t)  # 25 Hz (shaft speed)

def resample_signal(signal, original_fs, target_fs=10000, target_length=10000):
    num_samples = int(len(signal) * target_fs / original_fs)
    resampled = scipy.signal.resample(signal, num_samples)
    if len(resampled) >= target_length:
        return resampled[:target_length]
    return np.pad(resampled, (0, target_length - len(resampled)), 'constant')

def load_mfpt_with_simulation(base_dir):
    raw_data = []
    labels = []
    # class_map = {'Normal': 0, 'Bearing Fault': 1, 'Misalignment': 2, 'Rotor Imbalance': 3}
    
    subfolders = {
        'Three Baseline Conditions': 'Normal',
        'Three Outer Race Fault Conditions': 'Bearing Fault',
        'Seven More Outer Race Fault Conditions': 'Bearing Fault',
        'Seven Inner Race Fault Conditions': 'Bearing Fault'
    }
    
    mfpt_dir = os.path.join(base_dir, 'MFPT_Dataset')
    if not os.path.exists(mfpt_dir):
        raise FileNotFoundError(f"Directory '{mfpt_dir}' not found.")
    
    for subfolder, condition in subfolders.items():
        subfolder_path = os.path.join(mfpt_dir, subfolder)
        if not os.path.exists(subfolder_path):
            print(f"Warning: Subfolder '{subfolder_path}' not found. Skipping.")
            continue
        
        for file in os.listdir(subfolder_path):
            if file.endswith('.mat'):
                file_path = os.path.join(subfolder_path, file)
                mat = scipy.io.loadmat(file_path)
                
                signal = mat['bearing']['gs'][0][0].flatten()
                fs = mat['bearing']['sr'][0][0][0][0]
                signal_resampled = resample_signal(signal, fs)
                
                # Add original condition
                raw_data.append({
                    'file': file,
                    'subfolder': subfolder,
                    'signal': signal_resampled,
                    'original_signal': signal,
                    'sampling_rate': fs
                })
                labels.append(class_map[condition])
                
                # Simulate MA and RI from Normal signals
                if condition == 'Normal':
                    # Misalignment
                    ma_signal = simulate_misalignment(signal_resampled)
                    raw_data.append({
                        'file': f"{file}_MA_simulated",
                        'subfolder': subfolder,
                        'signal': ma_signal,
                        'original_signal': signal,
                        'sampling_rate': fs
                    })
                    labels.append(class_map['Misalignment'])
                    
                    # Rotor Imbalance
                    ri_signal = simulate_rotor_imbalance(signal_resampled)
                    raw_data.append({
                        'file': f"{file}_RI_simulated",
                        'subfolder': subfolder,
                        'signal': ri_signal,
                        'original_signal': signal,
                        'sampling_rate': fs
                    })
                    labels.append(class_map['Rotor Imbalance'])
    
    return raw_data, labels



In [9]:
!pip install --upgrade --force-reinstall "numpy<2.0"


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [10]:
!pip install --upgrade --force-reinstall contourpy gensim numba scipy


Defaulting to user installation because normal site-packages is not writeable
  Using cached contourpy-1.3.1-cp310-cp310-win_amd64.whl (218 kB)
  Using cached numba-0.61.0-cp310-cp310-win_amd64.whl (2.8 MB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl (41.2 MB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
  Using cached scipy-1.13.1-cp310-cp310-win_amd64.whl (46.2 MB)
  Using cached llvmlite-0.44.0-cp310-cp310-win_amd64.whl (30.3 MB)
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mario\\AppData\\Local\\Temp\\pip-uninstall-gg46335x\\scipy\\stats\\_ansari_swilk_statistics.cp310-win_amd64.pyd'
Check the permissions.

You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
if __name__ == "__main__":
    base_dir = os.getcwd()
    try:
        raw_data, labels = load_mfpt_with_simulation(base_dir)
        print(f"Loaded {len(raw_data)} signals:")
        for i, (data_entry, label) in enumerate(zip(raw_data, labels)):
            condition = [k for k, v in class_map.items() if v == label][0]
            print(f"Signal {i+1}: {data_entry['subfolder']}/{data_entry['file']}")
            print(f"  Condition: {condition}")
            print(f"  Original Sampling Rate: {data_entry['sampling_rate']} Hz")
            print(f"  Resampled Length: {len(data_entry['signal'])} samples")
            print()
        print(f"Label distribution: {np.bincount(labels)}")
    except Exception as e:
        print(f"Error: {e}")

Loaded 26 signals:
Signal 1: Three Baseline Conditions/baseline_1.mat
  Condition: Normal
  Original Sampling Rate: 97656 Hz
  Resampled Length: 10000 samples

Signal 2: Three Baseline Conditions/baseline_1.mat_MA_simulated
  Condition: Misalignment
  Original Sampling Rate: 97656 Hz
  Resampled Length: 10000 samples

Signal 3: Three Baseline Conditions/baseline_1.mat_RI_simulated
  Condition: Rotor Imbalance
  Original Sampling Rate: 97656 Hz
  Resampled Length: 10000 samples

Signal 4: Three Baseline Conditions/baseline_2.mat
  Condition: Normal
  Original Sampling Rate: 97656 Hz
  Resampled Length: 10000 samples

Signal 5: Three Baseline Conditions/baseline_2.mat_MA_simulated
  Condition: Misalignment
  Original Sampling Rate: 97656 Hz
  Resampled Length: 10000 samples

Signal 6: Three Baseline Conditions/baseline_2.mat_RI_simulated
  Condition: Rotor Imbalance
  Original Sampling Rate: 97656 Hz
  Resampled Length: 10000 samples

Signal 7: Three Baseline Conditions/baseline_3.mat
  

In [11]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mario\\AppData\\Roaming\\Python\\Python310\\site-packages\\~l_dtypes\\_ml_dtypes_ext.cp310-win_amd64.pyd'
Check the permissions.

You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [14]:
# preprocessing.py
import numpy as np
import scipy.signal
import tensorflow as tf

def resample_signal(signal, original_fs, target_fs=10000, target_length=10000):
    num_samples = int(len(signal) * target_fs / original_fs)
    resampled = scipy.signal.resample(signal, num_samples)
    if len(resampled) >= target_length:
        return resampled[:target_length]
    return np.pad(resampled, (0, target_length - len(resampled)), 'constant')

def add_awgn(signal, snr):
    signal_power = np.mean(signal**2)
    noise_power = signal_power / (10**(snr / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

def preprocess_signal(signal, fs=10000, snr=None):
    signal_resampled = resample_signal(signal, fs)
    if snr is not None:
        signal_resampled = add_awgn(signal_resampled, snr)
    f, t, Sxx = scipy.signal.spectrogram(signal_resampled, fs=fs, nperseg=256)
    Sxx = np.log1p(Sxx)
    Sxx_resized = tf.image.resize(Sxx[np.newaxis, :, :, np.newaxis], (224, 224)).numpy()[0, :, :, 0]
    Sxx_resized = (Sxx_resized - np.min(Sxx_resized)) / (np.max(Sxx_resized) - np.min(Sxx_resized))
    return Sxx_resized[..., np.newaxis]

def preprocess_data(raw_data, labels, snr_levels=[0, 10, 20]):
    X = []
    y = []
    for (signal, fs, condition, file), label in zip(raw_data, labels):
        for snr in snr_levels:
            spectrogram = preprocess_signal(signal, fs, snr)
            X.append(spectrogram)
            y.append(label)
    return np.array(X), np.array(y)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\mario\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\mario\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  F

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import